# Business Problem: Predict product failure for a transmission device based harware company.

In [1]:
#Importing the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing the dataset
#df=pd.read_csv('DeviceFailure.csv')
df = pd.read_csv('/content/drive/MyDrive/Dataset/DeviceFailure.csv')

FileNotFoundError: [Errno 2] File /content/drive/MyDrive/Dataset/DeviceFailure.csv does not exist: '/content/drive/MyDrive/Dataset/DeviceFailure.csv'

In [3]:
df = pd.read_csv('DeviceFailure.csv')

In [4]:
df.shape

(124494, 8)

In [12]:
# Checking the dataset loaded correctly
df.head()

,device,failure,attribute1,attribute2,attribute4,attribute5,attribute7,attribute9
0,S1F01085,0,215630672,56,52,6,0,7
1,S1F01085,0,1650864,56,52,6,0,7
2,S1F01085,0,124017368,56,52,6,0,7
3,S1F01085,0,128073224,56,52,6,0,7
4,S1F01085,0,97393448,56,52,6,0,7


In [13]:
df.tail()

,device,failure,attribute1,attribute2,attribute4,attribute5,attribute7,attribute9
124489,Z1F2PBHX,0,180917784,0,0,5,0,0
124490,Z1F2PBHX,0,33952520,0,0,5,0,0
124491,Z1F2PBHX,0,59053184,0,0,5,0,0
124492,Z1F2PBHX,0,110545440,0,0,5,0,0
124493,Z1F2PBHX,0,130522432,0,0,5,0,0


In [14]:
df['failure'].value_counts()

0    124388
1       106
Name: failure, dtype: int64

In [15]:
# Data Pre processing
x = df.loc[:, df.columns != "failure"]
y = df['failure']

In [16]:
x.head()

,device,attribute1,attribute2,attribute4,attribute5,attribute7,attribute9
0,S1F01085,215630672,56,52,6,0,7
1,S1F01085,1650864,56,52,6,0,7
2,S1F01085,124017368,56,52,6,0,7
3,S1F01085,128073224,56,52,6,0,7
4,S1F01085,97393448,56,52,6,0,7


In [17]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: failure, dtype: int64

In [18]:
# Splitting the Data
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=0)

In [19]:
# Removing the device column as it is not a real feature.
# but at the end we will use this data associating the prediction from user it came from

train_identifier = x_train['device']
x_train = x_train.drop(columns=['device'])

test_identifier = x_test['device']
x_test = x_test.drop(columns=['device'])

In [20]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [21]:
# scale the x_train and create a new variable for scaled x_train
#Standard scaler returns a numpy array so we need to convert to DataFrame
#In array form 'index' and 'column names' are lost we need it so convert to Df

x_train_scaled= pd.DataFrame(sc.fit_transform(x_train))
x_test_scaled = pd.DataFrame(sc.transform(x_test))

In [22]:
# put x_train_scaled have the columns of original x_train set
x_train_scaled.columns = x_train.columns.values

# put x_test_scaled have the columns of original x_test set
x_test_scaled.columns = x_test.columns.values

# take the indexes also
x_train_scaled.index = x_train.index.values
x_test_scaled.index = x_test.index.values

## Balanicing the Dataset.

In [23]:
# Balanicing the Dataset.
# create the training df by remerging X_train and y_train
df_train = x_train_scaled.join(y_train)
df_train.sample(10)

,attribute1,attribute2,attribute4,attribute5,attribute7,attribute9,failure
25688,0.141837,-0.073271,-0.077923,-0.202428,-0.041268,-0.023940,0
80316,-0.986174,-0.069551,-0.077923,-0.265370,-0.041268,-0.065921,0
91494,-0.253533,-0.073271,4.998986,0.364055,-0.041268,-0.065921,0
122693,1.472664,-0.073271,-0.077923,-0.202428,-0.041268,-0.060674,0
109410,0.845466,-0.073271,-0.077923,-0.454198,-0.041268,-0.065921,0
3031,-0.128126,-0.073271,-0.077923,-0.580083,-0.041268,-0.065921,0
112669,-1.358294,-0.073271,-0.077923,3.385293,-0.041268,-0.065921,0
6614,1.216513,-0.073271,-0.077923,3.007639,-0.041268,-0.065921,0
86115,0.342715,-0.073271,-0.077923,-0.076543,-0.041268,-0.065921,0
117972,-1.626669,-0.073271,-0.077923,-0.391255,-0.041268,-0.065921,0


In [24]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df_train[df_train.failure==0]
df_minority = df_train[df_train.failure==1]

print(df_majority.failure.count())
print("-----------")
print(df_minority.failure.count())
print("-----------")
print(df_train.failure.value_counts())

99512
-----------
83
-----------
0    99512
1       83
Name: failure, dtype: int64


<b> UP-Sampling the Data

In [25]:
# UPSAMPLING
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=99512,    # to match majority class
                                 random_state=587) # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
# Display new class counts
df_upsampled.failure.value_counts()

1    99512
0    99512
Name: failure, dtype: int64

In [26]:
x_upsampled = df_upsampled.drop(['failure'], axis= 1)
y_upsampled = df_upsampled.failure

In [27]:
# Splitting the Data
from sklearn.model_selection import train_test_split

x_up_train,x_up_test,y_up_train,y_up_test= train_test_split(x_upsampled,y_upsampled,test_size=0.2,random_state=0)

In [28]:
x_up_train.head()

,attribute1,attribute2,attribute4,attribute5,attribute7,attribute9
64496,0.473767,-0.073271,-0.077923,-0.013600,-0.041268,-0.065921
23861,0.417016,-0.073271,-0.077923,-0.391255,10.918911,-0.065921
10716,0.293393,-0.073271,-0.077923,-0.013600,-0.041268,-0.065921
35081,-0.790088,-0.073271,-0.077923,-0.202428,-0.041268,-0.034435
22640,-1.024272,-0.073271,-0.077923,-0.517140,-0.041268,-0.065921


<b># Applying machine learning algorithms

In [29]:
# Applying machine learning algorithms
# Creating a function that will give the Following output when any model is run

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, x_up_train,x_up_test,y_up_train,y_up_test, train=True):
    if train:
        pred = clf.predict(x_up_train)
        clf_report = pd.DataFrame(classification_report(y_up_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_up_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_up_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(x_up_test)
        clf_report = pd.DataFrame(classification_report(y_up_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_up_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_up_test, pred)}\n")

In [30]:
# LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0)

clf_lr.fit(x_up_train ,y_up_train)
print_score(clf_lr, x_up_train,x_up_test,y_up_train,y_up_test, train=True)
print_score(clf_lr, x_up_train,x_up_test,y_up_train,y_up_test, train=False)

Train Result:
Accuracy Score: 76.83%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.695323      0.929328  0.768338       0.812325       0.812228
recall         0.955934      0.580428  0.768338       0.768181       0.768338
f1-score       0.805063      0.714563  0.768338       0.759813       0.759851
support    79676.000000  79543.000000  0.768338  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[76165  3511]
 [33374 46169]]

Test Result:
Accuracy Score: 76.94%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.694555      0.934230  0.769351      0.814392      0.814793
recall         0.958812      0.581151  0.769351      0.769982      0.769351
f1-score       0.805566      0.716557  0.769351      0.761061      

In [31]:
# DECISION TREE

from sklearn.tree import DecisionTreeClassifier
clf_tree= DecisionTreeClassifier(random_state=0)

clf_tree.fit(x_up_train ,y_up_train)
print_score(clf_tree, x_up_train,x_up_test,y_up_train,y_up_test, train=True)
print_score(clf_tree, x_up_train,x_up_test,y_up_train,y_up_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      1.000000      0.999987  0.999994       0.999994       0.999994
recall         0.999987      1.000000  0.999994       0.999994       0.999994
f1-score       0.999994      0.999994  0.999994       0.999994       0.999994
support    79676.000000  79543.000000  0.999994  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[79675     1]
 [    0 79543]]

Test Result:
Accuracy Score: 99.96%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      1.000000      0.999299  0.999648      0.999650      0.999649
recall         0.999294      1.000000  0.999648      0.999647      0.999648
f1-score       0.999647      0.999650  0.999648      0.999648     

In [32]:
# Random Forest
# Random Forest
from sklearn.ensemble import RandomForestClassifier
clf_rf= RandomForestClassifier(random_state=0)

clf_rf.fit(x_up_train,y_up_train)
print_score(clf_rf, x_up_train,x_up_est,y_up_train,y_up_test, train=True)
print_score(clf_rf, x_up_train,x_up_test,y_up_train,y_up_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      1.000000      0.999987  0.999994       0.999994       0.999994
recall         0.999987      1.000000  0.999994       0.999994       0.999994
f1-score       0.999994      0.999994  0.999994       0.999994       0.999994
support    79676.000000  79543.000000  0.999994  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[79675     1]
 [    0 79543]]

Test Result:
Accuracy Score: 99.96%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      1.000000      0.999249  0.999623      0.999625      0.999623
recall         0.999244      1.000000  0.999623      0.999622      0.999623
f1-score       0.999622      0.999625  0.999623      0.999623     

In [33]:
# SVM
from sklearn.svm import SVC

print("=======================Linear Kernel SVM==========================")
model = SVC(kernel='linear')
model.fit(x_up_train,y_up_train)
print_score(model, x_up_train,x_up_test,y_up_train,y_up_test, train=True)
print_score(model, x_up_train,x_up_test,y_up_train,y_up_test, train=False)

=======================Linear Kernel SVM==========================
Train Result:
Accuracy Score: 73.87%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.663379      0.943984  0.738668       0.803682       0.803564
recall         0.969966      0.506984  0.738668       0.738475       0.738668
f1-score       0.787899      0.659676  0.738668       0.723787       0.723841
support    79676.000000  79543.000000  0.738668  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[77283  2393]
 [39216 40327]]

Test Result:
Accuracy Score: 73.91%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.662170      0.948932  0.739053      0.805551      0.806030
recall         0.972525      0.507136  0.739053      0.739830      0.739053


# Down-Sampling the Data

In [34]:
# Down-Sampling the Data
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=83,     # to match minority class
                                 random_state=24) # reproducible results
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
# Display new class counts
df_downsampled.failure.value_counts()

1    83
0    83
Name: failure, dtype: int64

In [35]:
x_downsampled = df_downsampled.drop(['failure'], axis = 1)
y_downsampled = df_downsampled.failure

In [36]:
# Splitting the Data
from sklearn.model_selection import train_test_split

x_down_train,x_down_test,y_down_train,y_down_test= train_test_split(x_downsampled,y_downsampled,test_size=0.2,random_state=0)

<b># Applying machine learning algorithms

In [37]:
# Applying machine learning algorithms
# Creating a function that will give the Following output when any model is run

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, x_down_train,x_down_test,y_down_train,y_down_test, train=True):
    if train:
        pred = clf.predict(x_down_train)
        clf_report = pd.DataFrame(classification_report(y_down_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_down_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_down_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(x_down_test)
        clf_report = pd.DataFrame(classification_report(y_down_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_down_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_down_test, pred)}\n")

In [38]:
# LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0)

clf_lr.fit(x_down_train, y_down_train)
print_score(clf_lr, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(clf_lr, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

Train Result:
Accuracy Score: 77.27%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy   macro avg  weighted avg
precision   0.711111   0.904762  0.772727    0.807937      0.805002
recall      0.941176   0.593750  0.772727    0.767463      0.772727
f1-score    0.810127   0.716981  0.772727    0.763554      0.764965
support    68.000000  64.000000  0.772727  132.000000    132.000000
_______________________________________________
Confusion Matrix: 
 [[64  4]
 [26 38]]

Test Result:
Accuracy Score: 67.65%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy  macro avg  weighted avg
precision   0.583333   0.900000  0.676471   0.741667      0.760294
recall      0.933333   0.473684  0.676471   0.703509      0.676471
f1-score    0.717949   0.620690  0.676471   0.669319      0.663598
support    15.000000  19.000000  0.676471  34.000000     34.000000
______________________

In [39]:
# DECISION TREE

from sklearn.tree import DecisionTreeClassifier
clf_tree= DecisionTreeClassifier(random_state=0)

clf_tree.fit(x_down_train,y_down_train)
print_score(clf_tree, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(clf_tree, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
              0     1  accuracy  macro avg  weighted avg
precision   1.0   1.0       1.0        1.0           1.0
recall      1.0   1.0       1.0        1.0           1.0
f1-score    1.0   1.0       1.0        1.0           1.0
support    68.0  64.0       1.0      132.0         132.0
_______________________________________________
Confusion Matrix: 
 [[68  0]
 [ 0 64]]

Test Result:
Accuracy Score: 76.47%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy  macro avg  weighted avg
precision   0.705882   0.823529  0.764706   0.764706      0.771626
recall      0.800000   0.736842  0.764706   0.768421      0.764706
f1-score    0.750000   0.777778  0.764706   0.763889      0.765523
support    15.000000  19.000000  0.764706  34.000000     34.000000
_______________________________________________
Confusion Matrix: 
 [[12  3]

In [40]:
# Random Forest
# Random Forest
from sklearn.ensemble import RandomForestClassifier
clf_rf= RandomForestClassifier(random_state=0)

clf_rf.fit(x_down_train,y_down_train)
print_score(clf_rf, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(clf_rf, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
              0     1  accuracy  macro avg  weighted avg
precision   1.0   1.0       1.0        1.0           1.0
recall      1.0   1.0       1.0        1.0           1.0
f1-score    1.0   1.0       1.0        1.0           1.0
support    68.0  64.0       1.0      132.0         132.0
_______________________________________________
Confusion Matrix: 
 [[68  0]
 [ 0 64]]

Test Result:
Accuracy Score: 73.53%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy  macro avg  weighted avg
precision   0.666667   0.812500  0.735294   0.739583      0.748162
recall      0.800000   0.684211  0.735294   0.742105      0.735294
f1-score    0.727273   0.742857  0.735294   0.735065      0.735982
support    15.000000  19.000000  0.735294  34.000000     34.000000
_______________________________________________
Confusion Matrix: 
 [[12  3]

In [41]:
# SVM
from sklearn.svm import SVC

print("=======================Linear Kernel SVM==========================")
model = SVC(kernel='linear')
model.fit(x_down_train,y_down_train)
print_score(model, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(model, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

=======================Linear Kernel SVM==========================
Train Result:
Accuracy Score: 78.03%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy   macro avg  weighted avg
precision   0.719101   0.906977  0.780303    0.813039      0.810192
recall      0.941176   0.609375  0.780303    0.775276      0.780303
f1-score    0.815287   0.728972  0.780303    0.772129      0.773437
support    68.000000  64.000000  0.780303  132.000000    132.000000
_______________________________________________
Confusion Matrix: 
 [[64  4]
 [25 39]]

Test Result:
Accuracy Score: 67.65%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy  macro avg  weighted avg
precision   0.583333   0.900000  0.676471   0.741667      0.760294
recall      0.933333   0.473684  0.676471   0.703509      0.676471
f1-score    0.717949   0.620690  0.676471   0.669319      0.663598
support    15.000000  

### Performing Hyper Parameter tuning for DT for UP Sampled Data

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"), 
    "splitter":("best", "random"), 
    "max_depth":(list(range(1,10))), 
    "min_samples_split":[2, 3, 4], 
    "min_samples_leaf":list(range(1, 20)), 
}


tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="precision", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(x_up_train ,y_up_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

tree_clf = DecisionTreeClassifier(**best_params)
tree_clf.fit(x_up_train ,y_up_train)
print_score(clf_tree, x_up_train,x_up_test,y_up_train,y_up_test, train=True)
print_score(clf_tree, x_up_train,x_up_test,y_up_train,y_up_test, train=False)


Fitting 3 folds for each of 2052 candidates, totalling 6156 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 696 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 1396 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2296 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 3396 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 4696 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 6156 out of 6156 | elapsed:  8.8min finished


Best paramters: {'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'})
Train Result:
Accuracy Score: 85.12%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.926015      0.798702  0.851249       0.862358       0.862411
recall         0.763768      0.938876  0.851249       0.851322       0.851249
f1-score       0.837102      0.863135  0.851249       0.850119       0.850108
support    79676.000000  79543.000000  0.851249  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[60854 18822]
 [ 4862 74681]]

Test Result:
Accuracy Score: 85.03%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.925184      0.798297   0.85032      0.861740      0.861528
recall         0

### Performing Hyper Parameter tuning for DT for DOWN Sampled Data

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"), 
    "splitter":("best", "random"), 
    "max_depth":(list(range(1,10))), 
    "min_samples_split":[2, 3, 4], 
    "min_samples_leaf":list(range(1, 20)), 
}


tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="precision", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(x_down_train,y_down_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

clf_tree = DecisionTreeClassifier(**best_params)
clf_tree.fit(x_down_train,y_down_train)
print_score(clf_tree, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(clf_tree, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

Fitting 3 folds for each of 2052 candidates, totalling 6156 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 988 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 5788 tasks      | elapsed:   16.2s


Best paramters: {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'})
Train Result:
Accuracy Score: 53.03%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy   macro avg  weighted avg
precision   0.523077   1.000000  0.530303    0.761538      0.754312
recall      1.000000   0.031250  0.530303    0.515625      0.530303
f1-score    0.686869   0.060606  0.530303    0.373737      0.383226
support    68.000000  64.000000  0.530303  132.000000    132.000000
_______________________________________________
Confusion Matrix: 
 [[68  0]
 [62  2]]

Test Result:
Accuracy Score: 44.12%
_______________________________________________
CLASSIFICATION REPORT:
                   0     1  accuracy  macro avg  weighted avg
precision   0.441176   0.0  0.441176   0.220588      0.194637
recall      1.000000   0.0  0.441176   0.500000      0.441176
f1-score    0.612245   0.0  0.441176   0.306122

[Parallel(n_jobs=-1)]: Done 6156 out of 6156 | elapsed:   17.1s finished
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forest hyperparameter tuning for UP Sampled Data

In [45]:
n_estimators = [50,100]
max_features = ['auto', 'sqrt']
max_depth = [2, 3, 5]
max_depth.append(None)
min_samples_split = [2, 5, 7]
min_samples_leaf = [1, 2, 4]


params_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="precision", cv=3, verbose=1, n_jobs=-1)


rf_cv.fit(x_up_train,y_up_train)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(x_up_train,y_up_train)

print_score(clf_rf, x_up_train,x_up_test,y_up_train,y_up_test, train=True)
print_score(clf_rf, x_up_train,x_up_test,y_up_train,y_up_test, train=False)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed: 17.3min finished


Best parameters: {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Train Result:
Accuracy Score: 88.75%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.920555      0.859229  0.887513       0.889892       0.889918
recall         0.848436      0.926656  0.887513       0.887546       0.887513
f1-score       0.883025      0.891670  0.887513       0.887348       0.887344
support    79676.000000  79543.000000  0.887513  159219.000000  159219.000000
_______________________________________________
Confusion Matrix: 
 [[67600 12076]
 [ 5834 73709]]

Test Result:
Accuracy Score: 88.60%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.919675      0.857547  0.885994      0.888611      0.888507
recall        

### Performing Hyper Parameter tuning for DT for DOWN Sampled Data

In [46]:
n_estimators = [50, 100]
max_features = ['auto', 'sqrt']
max_depth = [2, 3, 5]
max_depth.append(None)
min_samples_split = [2, 5, 7]
min_samples_leaf = [1, 2, 4]


params_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="precision", cv=2, verbose=1, n_jobs=-1)


rf_cv.fit(x_down_train,y_down_train)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(x_down_train,y_down_train)

print_score(clf_rf, x_down_train,x_down_test,y_down_train,y_down_test, train=True)
print_score(clf_rf, x_down_train,x_down_test,y_down_train,y_down_test, train=False)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   33.7s


Best parameters: {'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
              0     1  accuracy  macro avg  weighted avg
precision   1.0   1.0       1.0        1.0           1.0
recall      1.0   1.0       1.0        1.0           1.0
f1-score    1.0   1.0       1.0        1.0           1.0
support    68.0  64.0       1.0      132.0         132.0
_______________________________________________
Confusion Matrix: 
 [[68  0]
 [ 0 64]]

Test Result:
Accuracy Score: 73.53%
_______________________________________________
CLASSIFICATION REPORT:
                   0          1  accuracy  macro avg  weighted avg
precision   0.666667   0.812500  0.735294   0.739583      0.748162
recall      0.800000   0.684211  0.735294   0.742105      0.735294
f1-score    0.727273   0.742857  0.735294   0.735065      0.735982
support    15.0000

[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:   37.4s finished


As per precision rate the best model will be: DT or RF with UP Sampled Data because precision rate for both is Good.